In [1]:
# Batch: run code on multiple cluster/session/animal


In [2]:
#modules to find path of all sessions
import glob
import os
import numpy as np
from IPython.display import clear_output, display, HTML
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
#run other notebooks
%run plotRat_documentation_1_GeneralBehavior.ipynb
%run plotRat_documentation_2_Spiking.ipynb
%run loadRat_documentation.ipynb
%run loadRawSpike_documentation.ipynb

## 1 -  Fix parameters

In [3]:
# INFO: all the default parameters for preprocessing (/!\ do not change here, it wont affect anything)
defaultParam={
    "binSize":0.25,
    "trialOffset":20., #max end of trial, in seconds (position will be cutted)
    "sigmaSmoothPosition":0.1,  #smooth the position
    "sigmaSmoothSpeed":0.3, #smooth the speed
    "positionDiffRange": [2.,5.], #min and max differences allowed between two consecutive positions
                                  #min to correct start, max to correct jumps
    "pawFrequencyRange":[2.,10.],
    "startAnalysisParams":[10,0.2,0.5],
    "cameraToTreadmillDelay":2., #seconds, usual time between camera start and treadmill start
    "nbJumpMax" : 100., #if jumps>nbJumpMax, trial is badly tracked
    
    #parameter to detect end of trial (first position minima)
    "endTrial_backPos":55,  # minima is after the animal went once to the back (after first time position>backPos)
    "endTrial_frontPos":30, # minima's position is in front of treadmill (position[end]<frontPos)
    "endTrial_minTimeSec":4, # minima is after minTimeSec seconds (time[end]>minTimeSec)
    }

In [4]:
#------------------------Check Path !----------------------
root="/data"
#---------------------------------------------------
#list of all animals
animalList=[os.path.basename(path) for path in sorted(glob.glob(root+"/Rat*"))]
print("Available list of animals: %s"%animalList)

Available list of animals: ['Rat001', 'Rat002', 'Rat004', 'Rat005', 'Rat010', 'Rat015', 'Rat017', 'Rat022', 'Rat027', 'Rat032', 'Rat039', 'Rat041', 'Rat042', 'Rat043', 'Rat044', 'Rat045', 'Rat046', 'Rat047', 'Rat048', 'Rat049', 'Rat050', 'Rat051', 'Rat053', 'Rat054', 'Rat055', 'Rat056', 'Rat060', 'Rat061', 'Rat062', 'Rat063', 'Rat064', 'Rat065', 'Rat066', 'Rat067', 'Rat068', 'Rat069', 'Rat074', 'Rat101', 'Rat103']


In [5]:
animalList = ["Rat070","Rat071","Rat072","Rat074","Rat075"]#,"Rat077","Rat078","Rat079","Rat081","Rat082","Rat084"]
#animalList = ["Rat085","Rat086","Rat087","Rat088","Rat089","Rat091","Rat092"]
#animalList = ["Rat077","Rat078","Rat079","Rat081","Rat082","Rat084"]
#animalList = ["Rat095","Rat096","Rat097","Rat098","Rat099","Rat100"]
animalList = ["Rat103"]
#animalList = ["Rat076"]#,"Rat078","Rat079","Rat081","Rat082","Rat084","Rat085","Rat086","Rat087","Rat088","Rat089","Rat091","Rat092"]
tagList=[]
print("List of animal to loop through: %s"%animalList)
redo = False
override = False

List of animal to loop through: ['Rat103']


In [6]:
#-------------------------Check data type--------------------------
# PARAMETERS (used if the pickles don't exist)
param={
    "goalTime":7,#needed for pavel data only
    "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
    "maxTrialDuration":20,
    "interTrialDuration":10,#None pavel
    "endTrial_frontPos":30,
    "endTrial_backPos":55, 
    "endTrial_minTimeSec":4,
    "cameraSamplingRate":25, #needed for new setup    

    "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
    "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
     "nbJumpMax":100,#200 pavel
    "binSize":0.25,
    #parameters used to preprocess (will override the default parameters)
}  
#-=======================================================================================
#                             Pavel Data Warnings
#-=======================================================================================
#------------------------------------
#("WARNING 1: rat001 and rat004 not only in DLS")
#---------------------------------------------
#("Warning 2: Intertrial period is not fixed in Pavel Data \n --> it seems to be 27s for good trials and 16 for bad trials by looking at some sessions")
#------------------------
#("WARNING 3: GoalTime change with rats!!! rat004 = 5s  !!! rat005 = 6s \n --> change goalTime if needed")
#----------------------
#param ["goalTime"] = 7.
#----------------------------------------------------

## 2 -  Loop through every session of several animals

In [7]:
#===================================================================================
''' Option A: Plot functions "one plot per session"  - BEHAVIOR'''
#
# one session = one folder = one experiment
#
# A plot is done in a "plot function", either defined here or in another .py file (ex: plotData.py). 
#
# You can do several plots at once (by giving a list of "plot function").
#
# For every session, the plots are saved in *RatXXX/Experiments/RatXXX_20.../nameOfPlot.png*  
#
# An html page is created or updated: *RatXXX/Analysis/all_nameOfPlot.html*
#===================================================================================

#-------------list of plot functions to do on each session--------------------
#those functions should take as arguments: data (+ optionnal keywords arguments)
#the functions can return False (then no plot will be saved)

plotFunctionList=[plot_session_behavior]

#---------------------------loop on animal-------------------------------------
errorSession=[]
for animal in animalList:
    print("Animal %s"%animal)
    print("**********")
    #----Get the list of all session
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+"/"+animal+"/Experiments/Rat*")]
    sessionList=sorted(sessionList)
    nbSession=len(sessionList)
    #----loop through sessions
    for index,session in enumerate(sessionList):
        #clear_output()
        print("Session %s/%s: %s"%(index+1,nbSession,session))
        #----------------------check for tag file-------------------------
        isTagIn = True
        if len(tagList)>0:
            isTagIn = False
            for tag in tagList:
                    if os.path.exists(root+"/"+animal+'/Experiments'+'/'+session+'/Analysis/'+tag):
                        isTagIn = True
            if not isTagIn: 
                print("Tag not found, session skipped")
                print("--------")
                continue

        #----load data for this session (add redoPreprocess=True to overwrite preprocess)
        try:
            sessionData=Data(root,animal,session,param=param,redoPreprocess=redo)
        except Exception as e:
            display(HTML("<b> An error happened, skip session. Error message: </b>"))
            display(HTML("<b>"+e.__class__.__name__+": "+str(e)+"</b>"))
            errorSession.append(session)
            continue            
        #----skip if training data or no break time
        if (not sessionData.hasBehavior):
            print("No Behavior, skipped")
            print("--------")
            continue  
        #---skip if file "NotToAnalyze"
        try:
            if "NotToAnalyze" in sessionData.emptyAnalysisFiles:
                print("Not to analyze, skipped")
                print("--------")
                continue 
        except:
            continue
        #----plot the data (add name="myNewName" to change the name of the html, add override=True to redo the plots)
        sessionData.plot_session_png_html(plotFunctionList,override=override)
        print("--------")
#---------------------------------------------------------------------------------------       
print("Done")
if errorSession:
    display(HTML("<b> An error happened for the following session(s):"))
    print("\n".join(errorSession))
#=======================================================================================

Animal Rat103
**********
Session 1/31: Rat103_2016_10_24_15_44
Preprocessing behavior data...
ready to read everything
No file /data/Rat103/Experiments/Rat103_2016_10_24_15_44/Rat103_2016_10_24_15_44.entrancetimes


Session 2/31: Rat103_2016_10_25_15_39
Preprocessing behavior data...
ready to read everything
No file /data/Rat103/Experiments/Rat103_2016_10_25_15_39/Rat103_2016_10_25_15_39.entrancetimes


Session 3/31: Rat103_2016_10_26_14_32
Preprocessing behavior data...
ready to read everything
No file /data/Rat103/Experiments/Rat103_2016_10_26_14_32/Rat103_2016_10_26_14_32.entrancetimes


Session 4/31: Rat103_2016_10_27_15_23
Preprocessing behavior data...
ready to read everything
No file /data/Rat103/Experiments/Rat103_2016_10_27_15_23/Rat103_2016_10_27_15_23.entrancetimes


Session 5/31: Rat103_2016_10_28_16_17
Preprocessing behavior data...
ready to read everything
No file /data/Rat103/Experiments/Rat103_2016_10_28_16_17/Rat103_2016_10_28_16_17.entrancetimes


...

Session 17/31: Rat103_2016_12_09_16_52
Preprocessing behavior data...
ready to read everything
No file /data/Rat103/Experiments/Rat103_2016_12_09_16_52/Rat103_2016_12_09_16_52.entrancetimes


Session 18/31: Rat103_2016_12_12_18_48
Preprocess behavior data loaded from /data/Rat103/Experiments/Rat103_2016_12_12_18_48/Analysis/preprocesseddata_binsize250ms_.p
No spike data
png already exists: /data/Rat103/Experiments/Rat103_2016_12_12_18_48/plot_session_behavior.png
Html updated: /data/Rat103/Analysis/all_plot_session_behavior.html
--------
Session 19/31: Rat103_2016_12_13_16_43
Preprocess behavior data loaded from /data/Rat103/Experiments/Rat103_2016_12_13_16_43/Analysis/preprocesseddata_binsize250ms_.p
No spike data
png already exists: /data/Rat103/Experiments/Rat103_2016_12_13_16_43/plot_session_behavior.png
Html updated: /data/Rat103/Analysis/all_plot_session_behavior.html
--------
Session 20/31: Rat103_2016_12_14_16_34
Preprocess behavior data loaded from /data/Rat103/Experiments/Rat103_2016_12_14_16_34/Analysis/preprocesseddata_binsize250ms_.p
No spike data
png already exists: /data/Rat103/Experiments/Rat103_2016_12_14_16_34/plot_session_behavior.png
Html updated: /data/

Session 27/31: Rat103_2017_01_03_14_42
Preprocessing behavior data...
behavior data loaded from /data/Rat103/Experiments/Rat103_2017_01_03_14_42/Analysis/rawbehaviordata.p


Session 28/31: Rat103_2017_01_04_14_20
Preprocessing behavior data...
ready to read everything
Lick time data successfully loaded
Behavior data loaded from text files: new setup (.behav_param)
Preprocessing done
No spike data
0.324223294204
Html updated: /data/Rat103/Analysis/all_plot_session_behavior.html
--------
Session 29/31: Rat103_2017_01_05_14_09
Preprocessing behavior data...
ready to read everything
Lick time data successfully loaded
Behavior data loaded from text files: new setup (.behav_param)
Preprocessing done
No spike data
0.310137410909
Html updated: /data/Rat103/Analysis/all_plot_session_behavior.html
--------
Session 30/31: Rat103_2017_01_06_14_41
Preprocessing behavior data...
ready to read everything
Lick time data successfully loaded
Behavior data loaded from text files: new setup (.behav_param)
Preprocessing done
No spike data
0.178832207636
Html updated: /data/Rat103/Analysis/all_plot_session_behavior.html
--------
Session 31/31: Rat103_2017_01_09_14_32
Preprocess

Rat103_2016_10_24_15_44
Rat103_2016_10_25_15_39
Rat103_2016_10_26_14_32
Rat103_2016_10_27_15_23
Rat103_2016_10_28_16_17
Rat103_2016_11_03_15_53
Rat103_2016_11_11_15_46
Rat103_2016_11_28_16_17
Rat103_2016_11_29_17_55
Rat103_2016_11_30_18_37
Rat103_2016_12_01_17_48
Rat103_2016_12_02_17_07
Rat103_2016_12_05_17_55
Rat103_2016_12_06_17_13
Rat103_2016_12_07_17_16
Rat103_2016_12_08_18_52
Rat103_2016_12_09_16_52
Rat103_2016_12_22_11_54
Rat103_2017_01_03_14_42


In [ ]:
#===================================================================================
''' Option B: Plot functions "one plot per cluster"  - SPIKES'''
#
# A plot is done in a "plot function", either defined here or in another .py file (ex: plotData.py).   
#
# You can do several plots at once (by giving a list of "plot function")
#
# For every session, the plots are saved in *RatXXX/Experiments/RatXXX_20.../plots/shankX_clusterX_nameOfPlot.png*  
#
# Two html pages are created or updated: 
#  - *RatXXX/Experiments/RatXXX_20.../nameOfPlot.html*: all plots for session ("session html")
#  - *RatXXX/Analysis/all_nameOfPlot.html*: links to every "session html" for this rat
#
#===================================================================================

#-------------list of plot functions to do on each session--------------------
#those functions should take as arguments: data,shank,cluster,group (+ optionnal keywords arguments)
#the functions can return False (then no plot will be saved)
animalList=["Rat025"]
plotFunctionList=[plot_raster_correlogram,plot_waveforms]

#------------------------------loop on animal----------------------------------------------  
for animal in animalList:
    print("animal %s"%animal)
    print("**********")

    #Get the list of all session
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+"/"+animal+"/Experiments/Rat*")]
    sessionList=sorted(sessionList)
    nbSession=len(sessionList)

    errorSession=[]
    
    #loop through sessions
    for index,session in enumerate(sessionList):
        print("Session %s/%s: %s"%(index+1,nbSession,session))
        #load data for this session (add redoPreprocess=True to overwrite preprocess)
        #----------------------check for tag file-------------------------
        isTagIn = True
        if len(tagList)>0:
            isTagIn = False
            for tag in tagList:
                    if os.path.exists(root+"/"+animal+'/Experiments'+'/'+session+'/Analysis/'+tag):
                        isTagIn = True
            if not isTagIn: 
                print("Tag not found, session skipped")
                print("--------")
                continue

        #----load data for this session (add redoPreprocess=True to overwrite preprocess)
        try:
            sessionData=Data(root,animal,session,param=param,redoPreprocess=redo)
        except Exception as e:
            display(HTML("<b> An error happened, skip session. Error message: </b>"))
            display(HTML("<b>"+e.__class__.__name__+": "+str(e)+"</b>"))
            errorSession.append(session)
            print("--------")
            continue
            
        #skip if training data or no break time
        if (not sessionData.hasBehavior):
            print("No behavior, skipped")
            print("--------")
            continue
            
        #skip if file "NotToAnalyze"
        if "NotToAnalyze" in sessionData.emptyAnalysisFiles:
            print("Not to analyze, skipped")
            print("--------")
            continue        
            
        if sessionData.hasSpike:
            #plot the data (add name="myNewName" to change the name of the html, add override=True to redo the plots)
            # groupList can be "Good", ["Good","MUA"],... No groupList -> do every group
            sessionData.plot_all_clusters_png_html(plotFunctionList,override=override,groupList="Good")
        print("--------")
        #---------------------------------------------------------------------------------------            
        print("Done")
        if errorSession:
            display(HTML("<b> An error happened for the following session(s):"))
            print("\n".join(errorSession))
        #=======================================================================================